### 1. mon_feature.pkl upload

In [ ]:
import pandas as pd
import numpy as np
import pickle

FILE_PATH = '/content/drive/MyDrive/machine learning/team project/data/mon_features.pkl'
LABEL_COLUMN = ['website_label', 'monitored_label']

FEATURES_VER3 = [
    'total_transmission_time', 'std_inter_packet_time', 'avg_outgoing_burst_size',
    'avg_incoming_burst_size', 'num_outgoing_packets', 'incoming_packet_ratio',
    'outgoing_packet_ratio', 'cumul_packets_10pct', 'cumul_packets_30pct',
    'outgoing_order_skew', 'incoming_order_skew', 'cumul_max',
    'num_incoming_first_30', 'outgoing_first_30', 'avg_incoming_order_first_30',
    'avg_outgoing_order_first_30'
]

features_df = pd.read_pickle(FILE_PATH)
X = features_df[FEATURES_VER3]
y = features_df[LABEL_COLUMN[0]]

print(X)
print(y)

       total_transmission_time  std_inter_packet_time  \
0                        10.14               0.041168   
1                        10.16               0.163930   
2                        11.11               0.066661   
3                        13.36               0.047809   
4                        10.64               0.038760   
...                        ...                    ...   
18995                    43.91               0.143962   
18996                    15.60               0.019465   
18997                    14.93               0.016411   
18998                    19.91               0.033281   
18999                    13.76               0.011074   

       avg_outgoing_burst_size  avg_incoming_burst_size  num_outgoing_packets  \
0                     1.551282                16.666667                 121.0   
1                     1.702128                 9.319149                  80.0   
2                     1.552632                16.315789                 

### 2. LightGBM

#### (1) install

In [ ]:
pip install lightgbm

In [ ]:
import pandas as pd
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report

#### (2) Split datatset

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

#### (3) LightGBM 모델 정의

In [ ]:
model = lgb.LGBMClassifier(
    objective="multiclass",
    num_class=len(np.unique(y)),
    n_estimators=500,
    learning_rate=0.03,
    num_leaves=40,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

#### (4) 모델 학습

In [ ]:
model.fit(X_train, y_train)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

LGBMClassifier(colsample_bytree=0.8, learning_rate=0.03, n_estimators=500,
               num_class=95, num_leaves=40, objective='multiclass',
               random_state=42, subsample=0.8)

#### (5) 예측

In [ ]:
y_pred = model.predict(X_test)

#### (6) 평가

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:", classification_report(y_test, y_pred))

Accuracy: 0.8742105263157894

Classification Report:               precision    recall  f1-score   support

           0       0.74      0.78      0.76        40
           1       0.88      0.90      0.89        40
           2       0.88      0.88      0.88        40
           3       0.90      0.90      0.90        40
           4       0.90      0.88      0.89        40
           5       0.85      0.85      0.85        40
           6       0.81      0.88      0.84        40
           7       0.86      0.80      0.83        40
           8       0.94      0.85      0.89        40
           9       0.81      0.88      0.84        40
          10       0.80      0.82      0.81        40
          11       0.92      0.90      0.91        40
          12       0.88      0.93      0.90        40
          13       0.76      0.72      0.74        40
          14       0.94      0.78      0.85        40
          15       0.90      0.93      0.91        40
          16       0.82     

#### (7) 하이퍼파라미터 튜닝

In [ ]:
model = lgb.LGBMClassifier(objective='multiclass', num_class=len(np.unique(y)), random_state=42)

lgbm_param_grid = {
    'n_estimators': [400, 500],
    'max_depth': [3, 5],
    'learning_rate': [0.1, 0.03],
    'num_leaves': [40, 50],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

lgbm_grid = GridSearchCV(estimator=model, param_grid=lgbm_param_grid,
                         scoring='accuracy', cv=3, verbose=1, n_jobs=-1)

lgbm_grid.fit(X_train, y_train)

print("LightGBM Best Params (Multiclass):", lgbm_grid.best_params_)
print("LightGBM Best Accuracy (CV):", lgbm_grid.best_score_)

y_pred_lgbm = lgbm_grid.predict(X_test)
print("LightGBM Test Accuracy:", accuracy_score(y_test, y_pred_lgbm))

Streaming output truncated to the last 5000 lines.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit